In [4]:
# import libraries
import mlflow
import mlflow.sklearn

# name the experiment and create ml folder
mlflow.set_experiment('AI_Project_Methodology')

2023/02/13 14:06:43 INFO mlflow.tracking.fluent: Experiment with name 'AI_Project_Methodology' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///Users/raphsmart/Desktop/EPITA/2ndSemester/AI_Project_Methodology/Project/mlruns/510111486839379242', creation_time=1676293603451, experiment_id='510111486839379242', last_update_time=1676293603451, lifecycle_stage='active', name='AI_Project_Methodology', tags={}>

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn
import sys
import os

In [6]:
# create a function to calculate the metrics models

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [9]:
data_df = pd.read_csv('data.csv')
data_df.head()

,Car,Model,Volume,Weight,CO2
0,Toyoty,Aygo,1000,790,99
1,Mitsubishi,Space Star,1200,1160,95
2,Skoda,Citigo,1000,929,95
3,Fiat,500,900,865,90
4,Mini,Cooper,1500,1140,105


In [13]:
# train the model

def train_model(alpha, l1_ratio):
    train, test = train_test_split(data_df)
    
#     drop unwanted column
    X_train = train.drop(["Car", "Model", "CO2"], axis = 1)
    X_test = test.drop(["Car", "Model", "CO2"], axis = 1)
    y_train = train[["CO2"]]
    y_test = test[["CO2"]]
    
#     start mlflow
    with mlflow.start_run():
        El = ElasticNet(alpha = alpha, l1_ratio = l1_ratio, random_state = 42)
        El.fit(X_train, y_train)
        
        predicted_qualities = El.predict(X_test)
        
#         print the matrics parameters
        (rmse, mae, r2) = eval_metrics(y_test, predicted_qualities)
        print("ElasticNet model (alpha = %f, l1_ratio = %f):" % (alpha, l1_ratio))
        print(" RMSE: %s" % rmse)
        print(" MAE: %s" % mae)
        print(" R2: %s " % r2)
        
#         log mlflow on the parameters
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        
#         save the model
        mlflow.sklearn.log_model(El, "model")

In [14]:
# train the model, run (0.5) argumments for the two parameters
train_model(0.5, 0.5)

ElasticNet model (alpha = 0.500000, l1_ratio = 0.500000):
 RMSE: 7.79258324193735
 MAE: 7.324586348218769
 R2: -0.17897234903747572 


/Users/raphsmart/opt/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [15]:
# run with different arguments
train_model(0.2, 0.2)

ElasticNet model (alpha = 0.200000, l1_ratio = 0.200000):
 RMSE: 7.307453916824723
 MAE: 6.473613125319627
 R2: -0.7932460623830326 


In [16]:
train_model(0.1, 0.1)

ElasticNet model (alpha = 0.100000, l1_ratio = 0.100000):
 RMSE: 6.241832832875338
 MAE: 5.097147467189065
 R2: -0.8262723646981518 


In [17]:
train_model(0.4, 0.1)

ElasticNet model (alpha = 0.400000, l1_ratio = 0.100000):
 RMSE: 6.698250902738894
 MAE: 6.015873623955027
 R2: 0.2661163615429253 


In [18]:
train_model(0.1, 0.4)

ElasticNet model (alpha = 0.100000, l1_ratio = 0.400000):
 RMSE: 7.648103672678025
 MAE: 6.771853977991792
 R2: -0.39680798137692164 
